In [68]:
import pandas as pd
import numpy as np
import re


In [72]:
df=pd.read_csv('sushi.csv')
df.head()

,Name,Rating,Reviews,Price,Location
0,Shikibu Restaurant,4.5,409,$$,"{'address1': '10834 Washington Blvd', 'address..."
1,Yama Sushi Sake Attitude,5.0,214,$$,"{'address1': '11709 National Blvd', 'address2'..."
2,Uzumaki,4.5,669,$$,"{'address1': '11060 Washington Blvd', 'address..."
3,Izakaya Osen - Los Angeles,4.5,1631,$$,"{'address1': '2903 Sunset Blvd', 'address2': '..."
4,Roll Call - Koreatown,4.5,532,$$,"{'address1': '3183 Wilshire Blvd', 'address2':..."


In [73]:
len(df)

1000

In [74]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with the 'Location' column

# Define a function to extract the number of dollar signs
def extract_dollar_signs(price):
    if isinstance(price, str):
        return price.count('$')
    else:
        return 0

# Define a function to extract the street name
def extract_address_info(location):
    # Handle the case when the value is not a string
    if not isinstance(location, str):
        return '', '', ''

    # Convert the string to a dictionary
    location_dict = eval(location)

    # Extract the street address from the 'address1' key
    address = location_dict.get('address1')

    # Extract the ZIP code
    zip_code = location_dict.get('zip_code')

    # Extract the city name
    city = location_dict.get('city')

    # Check if the address exists and extract the street name
    if address:
        street_name = address.split(' ')[1]
        return street_name, zip_code, city
    else:
        return '', zip_code, city


# Apply the functions to create new columns
df['Price'] = df['Price'].apply(extract_dollar_signs)
df[['Street Name', 'Zip Code', 'City']] = df['Location'].apply(extract_address_info).apply(pd.Series)

# Print the updated DataFrame
df.drop('Location', axis=1, inplace=True)
df


,Name,Rating,Reviews,Price,Street Name,Zip Code,City
0,Shikibu Restaurant,4.5,409,2,Washington,90232,CULVER CITY
1,Yama Sushi Sake Attitude,5.0,214,2,National,90064,Los Angeles
2,Uzumaki,4.5,669,2,Washington,90232,Culver City
3,Izakaya Osen - Los Angeles,4.5,1631,2,Sunset,90026,Los Angeles
4,Roll Call - Koreatown,4.5,532,2,Wilshire,90010,Los Angeles
...,...,...,...,...,...,...,...
995,Itacho Ramen & Sushi,5.0,1,0,S,90017,Los Angeles
996,Okamoto Kitchen,4.0,663,2,,91423,Los Angeles
997,Midori Sushi - Encino,3.5,1034,2,Ventura,91316,Encino
998,Reel Inn Malibu,3.5,1136,2,Pacific,90265,Malibu
